

# 第5章：格式化输出和代表LLM发言

- [Lesson](#lesson)
- [Exercises](#exercises)
- [Example Playground](#example-playground)


## 设置
运行以下设置单元格以加载您的 API 密钥并建立 get_completion 辅助函数。

In [ ]:
!pip install anthropic

# Import python's built-in regular expression library
import re
import anthropic

# Retrieve the API_KEY & MODEL_NAME variables from the IPython store
%store -r API_KEY
%store -r MODEL_NAME

client = anthropic.Anthropic(api_key=API_KEY)

# New argument added for prefill text, with a default value of an empty string
def get_completion(prompt: str, system_prompt="", prefill=""):
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        system=system_prompt,
        messages=[
          {"role": "user", "content": prompt},
          {"role": "assistant", "content": prefill}
        ]
    )
    return message.content[0].text

---

## 课程
LLM 可以以各种各样的方式格式化其输出。您只需要要求它这样做！
其中一种方式是使用 XML 标签将响应与任何其他多余的文本分开。您已经学过可以使用 XML 标签使您的提示对 Claude 更清晰、更易解析。事实证明，您也可以要求 LLM **使用 XML 标签使其输出对人类更清晰、更容易理解**。


### 示例
还记得我们在第2章中通过要求Claude完全跳过前言来解决的"诗歌前言问题"吗？事实证明，我们也可以通过**告诉LLM将诗歌放在XML标签中** 来达到类似的效果。

In [16]:
# Variable content
ANIMAL = "Rabbit"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a haiku about {ANIMAL}. Put it in <haiku> tags."

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

--------------------------- Full prompt with variable substutions ---------------------------
Please write a haiku about Rabbit. Put it in <haiku> tags.

------------------------------------- Claude's response -------------------------------------
<haiku>
Fluffy, twitching nose,
Hopping through the verdant grass,
Rabbit's gentle grace.
</haiku>



为什么我们要这样做呢？嗯，将输出放在XML标签中允许最终用户通过编写一个简短的程序来**提取XML标签之间的内容，从而可靠地获得诗歌且仅获得诗歌**。
这种技术的延伸是将第一个XML标签放在assistant轮次中。当您在assistant轮次中放入文本时，您基本上是在告诉LLM它已经说了一些东西，并且应该从那一点继续下去。这种技术被称为"speaking for LLLM"或"prefilling LLM's response"。
下面，我们用第一个<haiku>XML标签做了这件事。请注意Claude是如何直接从我们停下的地方继续的


In [17]:
# Variable content
ANIMAL = "Cat"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a haiku about {ANIMAL}. Put it in <haiku> tags."

# Prefill for Claude's response
PREFILL = "<haiku>"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN:")
print(PROMPT)
print("\nASSISTANT TURN:")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

--------------------------- Full prompt with variable substutions ---------------------------
USER TURN:
Please write a haiku about Cat. Put it in <haiku> tags.

ASSISTANT TURN:
<haiku>

------------------------------------- Claude's response -------------------------------------

Feline grace and poise,
Purring softly by the fire,
Captivating cat.
</haiku>




LLM 也擅长使用其他输出格式样式，特别是 JSON。如果您想要强制 JSON 输出（不是确定性的，但接近于此），您也可以用开括号 { 预填充 LLM 的响应。

In [6]:
# Variable content
ANIMAL = "Cat"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a haiku about {ANIMAL}. Use JSON format with the keys as \"first_line\", \"second_line\", and \"third_line\"."

# Prefill for Claude's response
PREFILL = "{"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

--------------------------- Full prompt with variable substutions ---------------------------
USER TURN
Please write a haiku about Cat. Use JSON format with the keys as "first_line", "second_line", and "third_line".

ASSISTANT TURN
{

------------------------------------- Claude's response -------------------------------------

  "first_line": "Feline grace and poise,",
  "second_line": "Purring softly by my side,",
  "third_line": "Captivating cat."
}



下面是一个在同一个提示中使用**多个输入变量和输出格式规范的示例，全部使用XML标签完成**。

In [18]:
# 第一个输入变量
EMAIL = "Hi Zack, just pinging you for a quick update on that prompt you were supposed to write."

# 第二个输入变量
ADJECTIVE = "olde english"

# 带变量占位符的的提示词模板
PROMPT = f"Hey Claude. Here is an email: <email>{EMAIL}</email>. Make this email more {ADJECTIVE}. Write the new version in <{ADJECTIVE}_email> XML tags."

#预填充LLM的响应（作为带变量的f-string） 
PREFILL = f"<{ADJECTIVE}_email>"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

--------------------------- Full prompt with variable substutions ---------------------------
USER TURN
Hey Claude. Here is an email: <email>Hi Zack, just pinging you for a quick update on that prompt you were supposed to write.</email>. Make this email more olde english. Write the new version in <olde english_email> XML tags.

ASSISTANT TURN
<olde english_email>

------------------------------------- Claude's response -------------------------------------

Dearest Zack,

I do humbly beseech thee for a swift update on the prompt which thou wast charged to compose. Pray, let me know the progress of this endeavor, for I await thy response with bated breath.

Sincerely,
Thy humble servant
</olde english_email>




#### 额外课程
如果您通过API调用LLM，您可以将结束XML标签传递给stop_sequences参数，让Claude在发出您想要的标签后停止采样。这可以通过消除LLM在已经给出您关心的答案之后的结论性言论来节省金钱和到最后token的时间。

---

## 练习
- [Exercise 5.1 - Steph Curry GOAT](#exercise-51---steph-curry-goat)
- [Exercise 5.2 - Two Haikus](#exercise-52---two-haikus)
- [Exercise 5.3 - Two Haikus, Two Animals](#exercise-53---two-haikus-two-animals)

### Exercise 5.1 - Steph Curry GOAT

被迫做出选择时，LLM将迈克尔·乔丹指定为有史以来最好的篮球运动员。我们能让 LLM 选择其他人吗？
更改 PREFILL 变量以迫使 **LLM 做出详细论证，说明有史以来最好的篮球运动员是斯蒂芬·库里**。尽量不要改变除 PREFILL 之外的任何内容，因为这是本练习的重点。

In [ ]:
# Prompt template with a placeholder for the variable content
PROMPT = f"Who is the best basketball player of all time? Please choose one specific player."

# Prefill for LLM's response
PREFILL = ""

# Get Claude's response
response = get_completion(PROMPT, prefill=PREFILL)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search("Warrior", text))

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(response)
print("\n------------------------------------------ GRADING ------------------------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))


### 练习 5.2 - 两首俳句
使用 XML 标签修改下面的 PROMPT，使 LLM 写两首关于该动物的俳句（Haiku），而不是只写一首。应该清楚地区分一首诗的结束和另一首的开始。

In [ ]:
# Variable content
ANIMAL = "cats"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a haiku about {ANIMAL}. Put it in <haiku> tags."

# Prefill for Claude's response
PREFILL = "<haiku>"

# Get Claude's response
response = get_completion(PROMPT, prefill=PREFILL)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(
        (re.search("cat", text.lower()) and re.search("<haiku>", text))
        and (text.count("\n") + 1) > 5
    )

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(response)
print("\n------------------------------------------ GRADING ------------------------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

❓ If you want a hint, run the cell below!

In [ ]:
from hints import exercise_5_2_hint; print(exercise_5_2_hint)

### 两首俳句，两种动物
修改下面的 PROMPT，使LLM产生关于两种不同动物的两首俳句。使用 {ANIMAL1} 作为第一个替换的占位符，使用 {ANIMAL2} 作为第二个替换的占位符。

In [ ]:
# First input variable
ANIMAL1 = "Cat"

# Second input variable
ANIMAL2 = "Dog"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a haiku about {ANIMAL1}. Put it in <haiku> tags."

# Get Claude's response
response = get_completion(PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search("tail", text.lower()) and re.search("cat", text.lower()) and re.search("<haiku>", text))

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(response)
print("\n------------------------------------------ GRADING ------------------------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

❓ If you want a hint, run the cell below!

In [ ]:
from hints import exercise_5_3_hint; print(exercise_5_3_hint)


### 恭喜！
如果您已经解决了到此为止的所有练习，您就可以进入下一章了。祝您提示工程愉快！